In [3]:
import timeit
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np
import copy
import matplotlib.pyplot as plt


def lmform(
    data_list,
    config,
    join,
    transfer=None):
    
    start = timeit.default_timer()
    
    np.random.seed(seed=config["seed"])
    
    convergence_parameters = {}
    convergence_parameters["count"] = 0
    convergence_parameters["score_vec"] = [10e6]
    
    main_parameters, main_code = initialise_lmform(data_list = data_list,
                                         config = config,
                                         join = join,
                                         transfer = transfer
                                        )
    
    if (config["verbose"]):
            print("Beginning lmform learning with:    Sample dimension reduction (config[i_dim]): " + str( config["i_dim"] ) + "    Feature dimension reduction (config[j_dim]): " + str( config["j_dim"] ) + "    Tolerance Threshold: " + str( config["tol"] ) + "   Maximum number of iterations: "  + str( config["max_iter"] ) + "   Verbose: ", config["verbose"])

            
    while True:
        prev_encode = main_code["encode"]
        
        for i in range(len(join["complete"]["data_list"])):
            internal_parameters = {}
            internal_parameters["beta"] = main_parameters["beta"][join["complete"]["beta"][i]]
            internal_parameters["intercept"] = main_parameters["intercept"][join["complete"]["data_list"][i]]
            
            internal_code = {}
            internal_code["encode"] = main_code["encode"][join["complete"]["code"][i]]
            internal_code["code"] = main_code["code"][join["complete"]["code"][i]]
            
            return_parameters, return_code  = update_set_lmform( 
                                        x = data_list[join["complete"]["data_list"][i]],
                                        main_parameters = internal_parameters,
                                        main_code = internal_code,
                                        config = config,
                                        fix = transfer["fix"]
                                        )

            main_parameters["beta"][join["complete"]["beta"][i]] = internal_parameters["beta"]
            main_parameters["intercept"][join["complete"]["data_list"][i]] = internal_parameters["intercept"]
            
            main_code["code"][join["complete"]["code"][i]] = internal_code["code"]
            main_code["encode"][join["complete"]["code"][i]] = internal_code["encode"]
            
        total_mae = 0
        for X in range(len(join["complete"]["data_list"])):      
            total_mae += torch.mean(torch.abs(main_code["encode"][join["complete"]["code"][X]] - prev_encode[join["complete"]["code"][X]]))

        # Check convergence
        convergence_parameters["score_vec"] += [total_mae]
        MSE = convergence_parameters["score_vec"][-1]
        prev_MSE = convergence_parameters["score_vec"][-2]
        
        if convergence_parameters["count"]>=1:
            if config["verbose"]:
                print("Iteration:   "+str(convergence_parameters["count"])+"   with Tolerance of:   "+str(abs(prev_MSE - MSE)))
            if convergence_parameters["count"] >= config["max_iter"]:
                break
            if abs(prev_MSE - MSE) < config["tol"]:
                break
        convergence_parameters["count"] += 1

    if (config["verbose"]):
        print("Learning has converged for lmform, beginning (if requested) dimension reduction")

    return_data = {}
    return_data["main_parameters"] = main_parameters
    return_data["main_code"] = main_code
    return_data["meta_parameters"] = {}
    return_data["meta_parameters"]["config"] = config
    return_data["meta_parameters"]["join"] = join
    return_data["convergence_parameters"] = convergence_parameters
    
    stop = timeit.default_timer()

    return_data["run_time"] = {}
    return_data["run_time"]["start"] = start
    return_data["run_time"]["stop"] = stop
    return_data["run_time"]["run_time"] = stop - start
    
    return return_data
               
               
def initialise_lmform(
    data_list,
    config,
    join,
    transfer
):

    main_code = {}
    main_code["code"] = {}
    main_code["encode"] = {}

    main_parameters = {}
    main_parameters["beta"] = {}
    main_parameters["intercept"] = {}
    
    for i in range(len(join["complete"]["data_list"])):
        main_code["code"][join["complete"]["code"][i]] = []
        main_code["encode"][join["complete"]["code"][i]] = []

        main_parameters["beta"][join["complete"]["beta"][i]] = []
        main_parameters["intercept"][join["complete"]["data_list"][i]] = []
    

    for i in range(len(join["complete"]["data_list"])):

        if main_parameters["beta"][join["complete"]["beta"][i]] == []:
            if not len(transfer["main_parameters"]["beta"][join["complete"]["beta"][i]]) == 0:
                main_parameters["beta"][join["complete"]["beta"][i]] = transfer["main_parameters"]["beta"][join["complete"]["beta"][i]]
            else:
                main_parameters["beta"][join["complete"]["beta"][i]] = initialise_parameters_lmform(x = data_list[join["complete"]["data_list"][i]], dim_main = config["j_dim"], seed_main = 1, type_main = config["init"]["beta"]).T

        if main_code["code"][join["complete"]["encode"][i]] == []:
            if not len(transfer["main_code"]["encode"][join["complete"]["encode"][i]]) == 0:
                main_code["encode"][join["complete"]["code"][i]] = transfer["main_code"]["encode"][join["complete"]["code"][i]]
            else:
                main_code["encode"][join["complete"]["code"][i]] = data_list[join["complete"]["data_list"][i]]@main_parameters["beta"][join["complete"]["beta"][i]]

        if main_code["code"][join["complete"]["code"][i]] == []:
            if not len(transfer["main_code"]["code"][join["complete"]["code"][i]]) == 0:
                main_code["code"][join["complete"]["code"][i]] = transfer["main_code"]["code"][join["complete"]["code"][i]]
            else:
                main_code["code"][join["complete"]["code"][i]] = data_list[join["complete"]["data_list"][i]]@torch.linalg.pinv((main_parameters["beta"][join["complete"]["beta"][i]]).T)

        if main_parameters["intercept"][join["complete"]["data_list"][i]] == []:
            if not len(transfer["main_parameters"]["intercept"][join["complete"]["data_list"][i]]) == 0:
                main_parameters["intercept"][join["complete"]["data_list"][i]] = transfer["main_parameters"]["intercept"][join["complete"]["data_list"][i]]
            else:
                main_parameters["intercept"][join["complete"]["data_list"][i]] = torch.mean(data_list[join["complete"]["data_list"][i]] - main_code["code"][join["complete"]["code"][i]]@(main_parameters["beta"][join["complete"]["beta"][i]].T),0)
        
    return main_parameters, main_code
               
               
               
def initialise_parameters_lmform(
                            x,
                            dim_main, 
                            seed_main,
                            type_main):
    if type_main == "SVD":
        svd = TruncatedSVD(n_components=dim_main, n_iter=2, random_state=seed_main)
        return svd.fit(x).components_

    
    if type_main == "rand":
        rand_data = np.random.randn(dim_main,x.shape[1])
        return rand_data
    
    
               
def update_set_lmform(x,main_parameters,main_code,config,fix):

    if not fix["code"]:
        main_code["code"] = (x - main_parameters["intercept"])@torch.linalg.pinv(main_parameters["beta"].T)
               
    if not fix["beta"]:
        main_parameters["beta"] = (torch.linalg.pinv(main_code["code"])@(x - main_parameters["intercept"])).T
               
    if not fix["intercept"]:
        main_parameters["intercept"] = torch.mean(x - (main_code["code"])@(main_parameters["beta"]).T,0)
    
    if not fix["encode"]:
        main_code["encode"] = (x - main_parameters["intercept"])@(main_parameters["beta"])

    return main_parameters, main_code

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
config = {}
config["i_dim"] = 1
config["init"] =  {}
config["init"]["alpha"] = "SVD"
config["init"]["beta"] = "SVD"
config["max_iter"] =  15
config["verbose"] =  False
config["seed"] =  1
config["tol"] =  1e-5
       
join = {}
join["complete"] = {}
join["complete"]["data_list"] = ["data_1"]
join["complete"]["alpha"] = ["data_1"]
join["complete"]["beta"] = ["data_1"]
join["complete"]["code"] = ["data_1"]
join["complete"]["encode"] = ["data_1"]

transfer = {}
transfer["fix"] = {}
transfer["fix"]["alpha"] = False
transfer["fix"]["beta"] = False
transfer["fix"]["code"] = False
transfer["fix"]["intercept"] = False
transfer["fix"]["encode"] = False

transfer["main_code"] = {}
transfer["main_code"]["code"] = {}
transfer["main_code"]["encode"] = {}

transfer["main_parameters"] = {}
transfer["main_parameters"]["alpha"] = {}
transfer["main_parameters"]["beta"] = {}
transfer["main_parameters"]["intercept"] = {}


for i in range(len(join["complete"]["data_list"])):
    transfer["main_code"]["code"][join["complete"]["code"][i]] = []
    transfer["main_code"]["encode"][join["complete"]["encode"][i]] = []

    transfer["main_parameters"]["alpha"][join["complete"]["alpha"][i]] = []
    transfer["main_parameters"]["beta"][join["complete"]["beta"][i]] = []
    transfer["main_parameters"]["intercept"][join["complete"]["data_list"][i]] = []


In [8]:
from torch.nn.functional import one_hot

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [21]:
import math
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.init as init


class summary_layer(nn.Module):
    def __init__(self, J_dim, common_dim):
        super(summary_layer, self).__init__()

        self.code_func_c = nn.Linear(J_dim,common_dim)
        self.code_func_I = nn.Linear(common_dim,common_dim)

        self.beta_I = torch.eye(J_dim,J_dim)
        self.beta_func1 = nn.Linear(J_dim,common_dim)
        
    def forward(self, x, init, gradient):

        B,D = x.shape

        beta = self.beta_func1(self.beta_I)
        code = (self.code_func_I((self.code_func_c(x))))
        
        if init:
        
            data_list = {}
            data_list["data_1"] = x

            if gradient:
                transfer["main_parameters"]["beta"]["data_1"] = beta
                transfer["main_code"]["encode"]["data_1"] = transfer["main_code"]["code"]["data_1"] = code

                transfer["fix"]["beta"] = False
                transfer["fix"]["code"] = False
                transfer["fix"]["encode"] = False
            else:
                transfer["main_parameters"]["beta"]["data_1"] = torch.randn(beta.shape)
                transfer["main_code"]["encode"]["data_1"] = transfer["main_code"]["code"]["data_1"] = torch.randn(code.shape)

                transfer["fix"]["beta"] = False
                transfer["fix"]["code"] = False
                transfer["fix"]["encode"] = False

                
            config["j_dim"] = beta.shape[1]

            lmform_model = lmform(
                data_list = data_list,
                config = config,
                join = join,
                transfer = transfer
            )

            beta = lmform_model["main_parameters"]["beta"]["data_1"]

        x = x@beta
        return x


class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()

        self.features = features
        # self.classifier = nn.Sequential(
        #     nn.Dropout(),
        #     nn.Linear(512, 512),
        #     nn.ReLU(True),
        #     nn.Dropout(),
        #     nn.Linear(512, 512),
        #     nn.ReLU(True),
        #     nn.Linear(512, 10),
        # )
        self.classifier = summary_layer(512,1000)
        self.output = summary_layer(1000,10)

        
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x,False,True)
        x = self.output(x,False,True)
        return x


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}


def vgg11():
    """VGG 11-layer model (configuration "A")"""
    return VGG(make_layers(cfg['A']))


def vgg11_bn():
    """VGG 11-layer model (configuration "A") with batch normalization"""
    return VGG(make_layers(cfg['A'], batch_norm=True))




In [22]:
config["j_dim"] = 10

import torch.optim as optim

net = vgg11()

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.001, weight_decay=0)

In [23]:
import time

start = time.time()

for epoch in range(5):  # loop over the dataset multiple times
    cousnt=0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        count=i
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        
            
    print(f'[{epoch + 1}, {count + 1:5d}] loss: {running_loss / 99:.10f}')
    running_loss = 0.0

    end = time.time()
    print(end - start)  
    net.init = False

print('Finished Training')

end = time.time()
print(end - start)

[1,  1563] loss: 76.1479275118
660.7372841835022
[2,  1563] loss: 20.6181715673
2716.354638338089
[3,  1563] loss: 15.7575139653
4795.04381775856
[4,  1563] loss: 13.0960033145
7003.925266742706
[5,  1563] loss: 11.3457923926
9946.449864387512
Finished Training
9946.450338840485


In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')   

Accuracy of the network on the 10000 test images: 68.94 %


In [25]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 85.4 %
Accuracy for class: car   is 76.4 %
Accuracy for class: bird  is 75.8 %
Accuracy for class: cat   is 35.3 %
Accuracy for class: deer  is 47.0 %
Accuracy for class: dog   is 54.2 %
Accuracy for class: frog  is 82.4 %
Accuracy for class: horse is 81.9 %
Accuracy for class: ship  is 80.4 %
Accuracy for class: truck is 70.6 %
